<a href="https://colab.research.google.com/github/erendagasan/Eren-Dagasan-Personal/blob/main/binance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install -q bta-lib
!pip install -q ta
!pip install -q yfinance

import yfinance as yf
import btalib
import numpy as np
import pandas as pd
from ta.trend import PSARIndicator
from ta.momentum import WilliamsRIndicator
from ta.trend import AroonIndicator
from ta.volume import VolumePriceTrendIndicator
from ta.trend import CCIIndicator
from ta.momentum import ROCIndicator
from ta.trend import ADXIndicator
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler
import random
import numpy as np
import warnings
import ast
from binance.client import Client
import pandas as pd
import datetime

today = datetime.today().date()
tomorrow = today + timedelta(days=1)

start="2020-01-01"
end = tomorrow.strftime("%Y-%m-%d")

balance = 100000

warnings.filterwarnings("ignore")
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_rows', 500)

def create_indicators(data):
  data["RSI"] = btalib.rsi(data["Close"], period=14).df

  data["SMA5"] = btalib.sma(data['Close'], period=5).df
  data["SMA9"] = btalib.sma(data['Close'], period=9).df
  data["SMA10"] = btalib.sma(data['Close'], period=10).df
  data["SMA14"] = btalib.sma(data['Close'], period=14).df
  data["SMA20"] = btalib.sma(data['Close'], period=20).df
  data["SMA21"] = btalib.sma(data['Close'], period=21).df
  data["SMA50"] = btalib.sma(data['Close'], period=50).df
  data["SMA200"] = btalib.sma(data['Close'], period=200).df

  data["EMA5"] = btalib.ema(data['Close'], period=5).df
  data["EMA14"] = btalib.ema(data['Close'], period=14).df
  data["EMA21"] = btalib.ema(data['Close'], period=21).df
  data["EMA50"] = btalib.ema(data['Close'], period=50).df

  data["STOCH-K"] = btalib.stoch(data['High'], data['Low'], data['Close']).df["k"]
  data["STOCH-D"] = btalib.stoch(data['High'], data['Low'], data['Close']).df["d"]

  data["MACD"] = btalib.macd(data['Close']).df["macd"]
  data["SIGNAL"] = btalib.macd(data['Close']).df["signal"]
  data["HISTOGRAM"] = btalib.macd(data['Close']).df["histogram"]

  data["BB-UPPER"] = btalib.bbands(data['Close']).df['top']
  data["BB-MID"] = btalib.bbands(data['Close']).df['mid']
  data["BB-LOWER"] = btalib.bbands(data['Close']).df['bot']

  data["STDEV"] = data["Close"].rolling(window=10).std()

  data["PSAR"] = PSARIndicator(data["High"], data["Low"], data["Close"]).psar()

  data["WILLIAMS"] = WilliamsRIndicator(data["High"], data["Low"], data["Close"]).williams_r()

  data["AROON"] = AroonIndicator(close=data["Close"], window=25).aroon_indicator()

  data["OBV"] = VolumePriceTrendIndicator(close=data['Close'], volume=data['Volume']).volume_price_trend()

  data['CCI'] = CCIIndicator(close=data['Close'], low=data["Low"], high=data["High"], window=14).cci()

  data['ROC'] = ROCIndicator(close=data['Close'], window=5).roc()

  data['BULL'] = data['High'] - (data['High'].rolling(13).max() + data['Low'].rolling(13).min()) / 2
  data['BEAR'] = data['Low'] - (data['High'].rolling(13).max() + data['Low'].rolling(13).min()) / 2

  adx_indicator = ADXIndicator(high=data['High'], low=data['Low'], close=data['Close'], window=14)
  data['ADX'] = adx_indicator.adx()
  data['+DI'] = adx_indicator.adx_pos()
  data['-DI'] = adx_indicator.adx_neg()

  data = data.dropna()
  data = data.reset_index()
  return data

def score(data, x):
  scaler = MinMaxScaler(feature_range=(0, 100))
  scores = []

  x = ast.literal_eval(x)

  for row in range(data.shape[0]):
    score = 0

    if data["RSI"][row] < 30:
      score+=x[0]
    elif data["RSI"][row] > 70:
      score-=x[1]

    if data["RSI"][row] >= 75:
      score+=x[58]
    elif row > 0 and data["RSI"][row-1] > 73 and data["RSI"][row] < 73:
      score-=x[59]

    if row > 0 and data["RSI"].iloc[row-1] > 63 and data["RSI"].iloc[row] < 63:
      score-=x[2]

    if row > 0 and data["RSI"].iloc[row-1] > 49 and data["RSI"].iloc[row] < 49:
      score-=x[3]
    elif row > 0 and data["RSI"].iloc[row-1] < 50 and data["RSI"].iloc[row] > 50:
      score+=x[4]

    if data["SMA5"].iloc[row] > data["SMA10"].iloc[row]:
      score+=x[5]
    else:
      score-=x[6]

    if row > 0 and data['SMA5'].iloc[row-1] < data['SMA10'].iloc[row-1] and data['SMA5'].iloc[row] > data['SMA10'].iloc[row]:
      score+=x[7]
    else:
      score-=x[8]

    if data["SMA9"][row] > data["SMA14"][row]:
      score+=x[9]
    else:
      score-=x[10]

    if row > 0 and data['SMA9'].iloc[row-1] < data['SMA14'].iloc[row-1] and data['SMA9'].iloc[row] > data['SMA14'].iloc[row]:
      score+=x[11]
    else:
      score-=x[12]

    if data["SMA14"][row] > data["SMA21"][row]:
      score+=x[13]
    else:
      score-=x[14]

    if row > 0 and data['SMA14'].iloc[row-1] < data['SMA21'].iloc[row-1] and data['SMA14'].iloc[row] > data['SMA21'].iloc[row]:
      score+=x[15]
    else:
      score-=x[16]

    if row > 1 and data['SMA20'].iloc[row-1] < data["Close"].iloc[row-1] and data['SMA20'].iloc[row] < data["Close"].iloc[row]:
      score+=x[17]
    else:
      score-=x[18]

    if data["SMA21"][row] > data["SMA50"][row]:
      score+=x[19]
    else:
      score-=x[20]

    if row > 0 and data['SMA21'].iloc[row-1] < data['SMA50'].iloc[row-1] and data['SMA21'].iloc[row] > data['SMA50'].iloc[row]:
      score+=x[21]
    else:
      score-=x[22]

    if row > 0 and data['SMA50'].iloc[row-1] < data['SMA200'].iloc[row-1] and data['SMA50'].iloc[row] > data['SMA200'].iloc[row]:
      score+=x[23]

    if data['Close'][row] > data["EMA14"][row]:
      score+=x[24]
    else:
      score-=x[25]

    if data["EMA5"][row] > data["EMA14"][row]:
      score+=x[26]
    else:
      score-=x[27]

    if row > 0 and data['EMA5'].iloc[row-1] < data['EMA14'].iloc[row-1] and data['EMA5'].iloc[row] > data['EMA14'].iloc[row]:
      score+=x[28]
    else:
      score-=x[29]

    if data["EMA14"][row] > data["EMA21"][row]:
      score+=x[30]
    else:
      score-=x[31]

    if row > 0 and data['EMA14'].iloc[row-1] < data['EMA21'].iloc[row-1] and data['EMA14'].iloc[row] > data['EMA21'].iloc[row]:
      score+=x[32]
    else:
      score-=x[33]

    if data["EMA21"][row] > data["EMA50"][row]:
      score+=x[34]
    else:
      score-=x[35]

    if row > 0 and data['EMA21'].iloc[row-1] < data['EMA50'].iloc[row-1] and data['EMA21'].iloc[row] > data['EMA50'].iloc[row]:
      score+=x[36]
    else:
      score-=x[37]

    if data["STOCH-D"][row] <= 20:
      score+=x[38]
    elif data["STOCH-D"][row] >= 80:
      score-=x[39]

    if row > 0 and data["STOCH-D"][row-1] > 65 and data["STOCH-D"][row] < 65:
      score-=x[40]
    elif row > 0 and data["STOCH-D"][row-1] < 35 and data["STOCH-D"][row] > 35:
      score=+x[41]

    if data["MACD"][row] > data["SIGNAL"][row]:
      score+=x[42]
    if row > 0 and data["MACD"][row] > data["SIGNAL"][row] and data["HISTOGRAM"][row-1] < 0 and data["HISTOGRAM"][row] > 0:
      score+=x[43]
    if row > 0 and data["MACD"][row] < data["SIGNAL"][row] and data["HISTOGRAM"][row-1] > 0 and data["HISTOGRAM"][row] < 0:
      score-=x[44]
    if data["MACD"][row] < data["SIGNAL"][row]:
      score-=x[45]

    if data['MACD'].tail(7).mean() > 0:
      score+=x[46]
    else:
      score-=x[47]

    if data['MACD'].tail(30).mean() > 0:
      score+=x[48]
    else:
      score-=x[49]

    if data["BB-LOWER"].iloc[row] > data["Close"].iloc[row]:
        score+=x[50]
    elif data["BB-UPPER"].iloc[row] < data["Close"].iloc[row]:
        score-=x[51]
    if data["BB-MID"].iloc[row] < data["Close"].iloc[row] and data["BB-LOWER"].iloc[row] - data["Close"].iloc[row] < data["BB-MID"].iloc[row] - data["Close"].iloc[row]:
        score+=x[52]
    elif data["BB-MID"].iloc[row] > data["Close"].iloc[row] and data["BB-UPPER"].iloc[row] - data["Close"].iloc[row] < data["BB-MID"].iloc[row] - data["Close"].iloc[row]:
        score-=x[53]

    if (data["STDEV"].max() + data["STDEV"].min()) / 2 > data["STDEV"][row]:
      score+=1
    else:
      score-=1

    if data["PSAR"][row] < data["Close"][row]:
      score+=1
    else:
      score-=1

    if data["WILLIAMS"][row] < -80:
      score+=1
    elif data["WILLIAMS"][row] > -20:
      score-=1

    if data["AROON"][row] > 0:
      score-=1
    else:
      score+=1

    if row > 0 and data["OBV"][row-1] < data["OBV"][row] and data["Close"][row] > data["Close"][row-1]:
      score+=1
    elif row > 0 and data["OBV"][row-1] > data["OBV"][row] and data["Close"][row] < data["Close"][row-1]:
      score-=1

    if row > 0 and (data['CCI'][row] > -100) & (data['CCI'][row-1] <= -100):
      score+=1
    elif row > 0 and (data['CCI'][row] < 100) & (data['CCI'][row-1] >= 100):
      score-=1

    if data["ROC"][row] > 0:
      score+=1
    elif data["ROC"][row] < 0:
      score-=1

    if data["BULL"][row] > 0:
      score+=1
    elif data["BEAR"][row] < 0:
      score-=1

    if (data['ADX'][row] > 25) and (data['+DI'][row] > data['-DI'][row]):
      score+=x[54]
    if data['ADX'][row] > data['+DI'][row] and data['ADX'][row] > data['-DI'][row]:
      score+=x[55]
    if data['ADX'][row] < data['+DI'][row] and data['ADX'][row] < data['-DI'][row]:
      score-=x[56]
    elif (data['ADX'][row] > 40) and (data['+DI'][row] < data['-DI'][row]):
      score-=x[57]

    scores.append(score)

  data["SCORE"] = scores
  data["SCORE"] = scaler.fit_transform(data["SCORE"].values.reshape(-1, 1))
  return data

In [7]:
x = str([3, 0, 0, 3, 1, 5, 0, 4, 4, 2, 5, 2, 3, 5, 4, 5, 1, 1, 1, 2, 1, 2, 1, 5, 0, 1, 3, 2, 1, 3, 0, 1, 2, 3, 4, 4, 1, 0, 1, 5, 5, 5, 2, 1, 5, 4, 4, 1, 3, 3, 5, 4, 5, 4, 2, 0, 5, 4, 1, 4])
buy_threshold = 46
sell_threshold = 57

In [ ]:
data = yf.download("BTC-USD", start=start, end="2023-06-19", progress=False)
data = create_indicators(data)
data = score(data, x)

if data["SCORE"].iloc[-1] >= buy_threshold and data["SCORE"].iloc[-2] < buy_threshold:
  print("BUY")
elif data["SCORE"].iloc[-1] <= sell_threshold and data["SCORE"].iloc[-2] > sell_threshold:
  print("SELL")

data.tail(50)

In [ ]:
!pip install python-binance

In [25]:
from binance.client import Client
import pandas as pd
import datetime

api_key = "xyFem5lVB7S2qX83o43kpH5YVxfExPXgSUActNuOchvF61Ax8Plfis5fcOT39kMi"
api_secret = "4J0MJBjVKB8NDKQzRXndgOKrlwcGKKkEIzR1wur87gTPioZTZRHncSB5UkxMi9Mu"

client = Client(api_key, api_secret)

account_info = client.get_account()
balances = account_info['balances']

for balance in balances:
    asset = balance['asset']
    free = float(balance['free'])
    locked = float(balance['locked'])
    total = free + locked
    print(f"Asset: {asset}, Free: {free:.8f}, Locked: {locked:.8f}, Total: {total:.8f}")

symbol = "BTCUSDT"
interval = Client.KLINE_INTERVAL_1DAY

two_years_ago = datetime.datetime.now() - datetime.timedelta(days=365*2)
timestamp_two_years_ago = int(two_years_ago.timestamp() * 1000)
klines = client.get_historical_klines(symbol, interval, timestamp_two_years_ago)
columns = ["timestamp", "open", "high", "low", "close", "volume", "close_time", "quote_asset_volume", "number_of_trades", "taker_buy_base_asset_volume", "taker_buy_quote_asset_volume", "ignore"]
df = pd.DataFrame(klines, columns=columns)

df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")
df

BinanceAPIException: ignored